In [ ]:
import os

ROOT_DIR = os.getcwd()

print(ROOT_DIR)

DATA_DIR = os.path.join(ROOT_DIR,'data')
TRAINING_DIR = os.path.join(ROOT_DIR,'training')

force_create = False

# crete dir structure if it doesnt exist
if force_create or not os.path.exists(ROOT_DIR):

    os.makedirs(ROOT_DIR)
    os.makedirs(DATA_DIR)
    os.makedirs(TRAINING_DIR)

    print(f"Data folder structure '{ROOT_DIR}' created successfully.")
else:
    print(f"Data folder structure '{ROOT_DIR}' already exists.")

def delete_and_recreate_folder(folder_path, delete=False):
    if delete:
        # Check if the folder exists
        if os.path.exists(folder_path):
            try:
                # Delete the folder and its contents
                shutil.rmtree(folder_path)
                print(f"Folder {folder_path} and its contents were deleted.")
            except Exception as e:
                print(f"Failed to delete {folder_path}. Reason: {e}")
                return
        else:
            print(f"Folder {folder_path} does not exist.")
        
        try:
            os.makedirs(folder_path)
            print(f"Folder {folder_path} was recreated successfully.")
        except Exception as e:
            print(f"Failed to create folder {folder_path}. Reason: {e}")
    else:
        print(f"Deletion skipped. Folder {folder_path} was not modified.")
import yaml

def init_yaml(kfold_number=1):
        
    data = {
        'path': f'{DATA_DIR}/{kfold_number}',
        'train': 'train/images',
        'val': 'val/images',
    
        'names': {
            0: 'wound'
        }
    }

    with open(os.path.join(ROOT_DIR, "config.yaml"), "w") as f:
        yaml.dump(data, f, default_flow_style=False)

In [ ]:
import torch.multiprocessing as mp
from multiprocessing import Queue

def train_model_function(i, result_queue):
    model = YOLO('yolov8x-seg.pt')
    results_training = model.train(data=os.path.join(ROOT_DIR, "config.yaml"), project=TRAINING_DIR, name="train_test_" + str(i),
                                   epochs=1, imgsz=512, batch=16, deterministic=True, plots=True, seed=1401,
                                   close_mosaic=0, augment=False, hsv_h=0, hsv_s=0, hsv_v=0, degrees=0, translate=0, scale=0, 
                                   shear=0.0, perspective=0, flipud=0, fliplr=0, bgr=0, mosaic=0, mixup=0, copy_paste=0, erasing=0,crop_fraction=0)
    r = {
        'train_map': results_training.seg.map,
        'train_map50': results_training.seg.map50,
        'train_map75': results_training.seg.map75,
        'train_precision': float(results_training.seg.p[0]),
        'train_recall': float(results_training.seg.r[0]),
        'train_f1': float(results_training.seg.f1[0]),
    }
    result_queue.put((r))

def start_training_in_process(i):
    #mp.set_start_method("spawn", force=True)  # Use 'spawn' for new processes
    
    result_queue = Queue()
    train_process = mp.Process(
        target=train_model_function,
        args=(i, result_queue)
    )
    train_process.name = "TrainingProcess"
    train_process.start()
    train_process.join()

    if not result_queue.empty():
        r = result_queue.get()
        return r
    else:
        print("No results returned from process")
        return None

In [ ]:
from ultralytics import YOLO


if __name__ == '__main__':

   #loop through different kfolds of dataset
    for i in range(1,6):
        init_yaml(i)
        start_training_in_process(i)